In [2]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
import ConnectionConfig as cc

cc.setupEnvironment()

In [3]:
spark = cc.startLocalCluster("FACT_RIT", 4)
spark.getActiveSession()

In [4]:
# Extract
cc.set_connectionProfile("velodb")

df_stations = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "stations") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "stationid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

In [5]:
df_rides = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "rides") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 100) \
    .option("lowerBound", 0) \
    .option("upperBound", 4138255) \
    .load()

In [6]:
df_stations.createOrReplaceTempView("stations")
df_rides.createOrReplaceTempView("rides")

In [7]:
df_stations = df_stations.select("zipcode", "gpscoord", "stationid", "zipcode")
df_zipcodes = df_stations.select("zipcode").distinct()
df_rides = df_rides.select("startpoint", "starttime")

In [17]:
import requests
import json

base_url = "http://api.openweathermap.org/data/2.5/weather"
api_key = "b6c3cf9ed2f94044caaa7376dfcfef9a"

zipcodes_calls = {} 

for row in df_stations.distinct().collect():

    zip = row["zipcode"]+",be"

    if zipcodes_calls.get(zip, 0) < 3:
        params = {
            "zip": zip,
            "appid": api_key
        }

        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            data = json.loads(response.text)
            print(data)
            zipcodes_calls[zip] = zipcodes_calls.get(zip, 0) + 1
            with open(f"./FileStore/weather/weather_{row['zipcode']}_{zipcodes_calls[zip]}.json", "w") as f:
                json.dump(data, f)
        else:
            print("Error")
            print(response.text)
            break

{'coord': {'lon': 4.4167, 'lat': 51.2167}, 'weather': [{'id': 300, 'main': 'Drizzle', 'description': 'light intensity drizzle', 'icon': '09d'}, {'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 285.7, 'feels_like': 285.49, 'temp_min': 285.11, 'temp_max': 286.46, 'pressure': 1008, 'humidity': 95, 'sea_level': 1008, 'grnd_level': 1007}, 'visibility': 8000, 'wind': {'speed': 4.12, 'deg': 130}, 'rain': {'1h': 0.34}, 'clouds': {'all': 75}, 'dt': 1727689398, 'sys': {'type': 2, 'id': 2006860, 'country': 'BE', 'sunrise': 1727674896, 'sunset': 1727716963}, 'timezone': 7200, 'id': 0, 'name': 'Antwerpen', 'cod': 200}
{'coord': {'lon': 4.4167, 'lat': 51.2167}, 'weather': [{'id': 300, 'main': 'Drizzle', 'description': 'light intensity drizzle', 'icon': '09d'}, {'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 285.7, 'feels_like': 285.49, 'temp_min': 285.11, 'temp_max': 286.46, '